In [2]:
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from glob import glob
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import optimizers, callbacks
import keras

# GPU使用のおまじない
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


Using TensorFlow backend.


In [3]:
# 学習済みモデル読み込み
# base_model = InceptionV3(weights='imagenet', include_top=False)
# base_model = InceptionResNetV2(weights='imagenet', include_top=False)
base_model = ResNet50(weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
INPUT_SIZE = 224
SEED = 1987
data_dir = '../input/dog-breed-identification'
labels = pd.read_csv(join(data_dir, 'labels.csv'))
sample_submission = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))), len(sample_submission))

NUM_CLASSES = labels['breed'].unique().size

10222 10222
10357 10357


In [15]:
# average pooling 追加
x = base_model.output
x = GlobalAveragePooling2D()(x)
# 全結合相追加
x = Dense(1024, activation='relu')(x)
# 出力層追加
predictions = Dense(120, activation='softmax')(x)

In [16]:
# モデルの結合
model = Model(inputs=base_model.input, outputs=predictions)

In [5]:
# 事前学習モデルのレイヤーの重みを固定する場合
# for layer in base_model.layers:
#     layer.trainable = False

In [17]:
# 最適化関数を指定
model.compile(optimizer=optimizers.adam(lr=1e-4), loss='categorical_crossentropy')
# model.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy')

In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [19]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

NUM_TRAINING = 20000
NUM_VALIDATION = 500

In [ ]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

In [38]:
for i, (img_id, breed, _) in labels.iterrows():
    print(img_id, breed)
    
    if i == 3:
        break

000bec180eb18c7604dcecc8fe0dba07 boston_bull
001513dfcb2ffafc82cccf4d8bbaba97 dingo
001cdf01b096e06d78e9e5112d419397 pekinese
00214f311d5d2247d5dfe4fe24b2303d bluetick


In [35]:
for i,ii in labels.iterrows():
    print(list(ii))
    break

['000bec180eb18c7604dcecc8fe0dba07', 'boston_bull', 1]


In [36]:
labels.head

<bound method NDFrame.head of                                      id                        breed  target
0      000bec180eb18c7604dcecc8fe0dba07                  boston_bull       1
1      001513dfcb2ffafc82cccf4d8bbaba97                        dingo       1
2      001cdf01b096e06d78e9e5112d419397                     pekinese       1
3      00214f311d5d2247d5dfe4fe24b2303d                     bluetick       1
4      0021f9ceb3235effd7fcde7f7538ed62             golden_retriever       1
5      002211c81b498ef88e1b40b9abf84e1d           bedlington_terrier       1
6      00290d3e1fdd27226ba27a8ce248ce85           bedlington_terrier       1
7      002a283a315af96eaea0e28e7163b21b                       borzoi       1
8      003df8b8a8b05244b1d920bb6cf451f9                      basenji       1
9      0042188c895a2f14ef64a918ed9c7b64           scottish_deerhound       1
10     004396df1acd0f1247b740ca2b14616e            shetland_sheepdog       1
11     0067dc3eab0b3c3ef0439477624d85d6       

In [ ]:
train_datagen = image.ImageDataGenerator(
   rescale=1.0 / 255,
   zoom_range=0.2,
   horizontal_flip=True,
   rotation_range=10)

test_datagen = image.ImageDataGenerator(
   rescale=1.0 / 255,
)

train_generator = train_datagen.flow(
   x,
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   shuffle=True
)

val_generator = test_datagen.flow_from_directory(
   'images/test',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

In [ ]:
# add call back
tb_cb = callbacks.TensorBoard(log_dir="./tflog4", write_images=True)
mc_cb = callbacks.ModelCheckpoint('models/weights.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss',save_best_only=True, period=1)
es_cb = callbacks.EarlyStopping(monitor='val_loss', patience=5)
cbks = [tb_cb, mc_cb, es_cb]

In [ ]:
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=NUM_TRAINING//BATCH_SIZE,
    epochs=10000,
    verbose=1,
    validation_data=val_generator,
    validation_steps=NUM_VALIDATION//BATCH_SIZE,
    callbacks=cbks,
    use_multiprocessing=True
   )

model.save('finished_model.h5')

In [ ]:

from keras.applications.resnet50 import preprocess_input, decode_predictions

img_path = 'images/predict/chihuahua.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

In [ ]:
img

In [ ]:
model = keras.models.load_model('models/weights.02-0.81.h5', compile=False)

In [ ]:
preds = model.predict(x)

In [ ]:
list(preds[0] > 0)

In [ ]:
[k for i, k in enumerate(train_generator.class_indices.keys()) if list(preds[0] > 0)[i]]